In [1]:
import yfinance as yf

In [27]:
aapl = yf.download(tickers="AAPL", period="7d", interval="1m")
aapl

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-14 09:30:00-04:00,172.320007,173.050003,172.050003,172.320007,172.320007,2039060
2024-03-14 09:31:00-04:00,172.309998,172.554993,172.240005,172.429993,172.429993,194477
2024-03-14 09:32:00-04:00,172.440002,172.520004,172.199997,172.479996,172.479996,215352
2024-03-14 09:33:00-04:00,172.485001,172.589996,172.389999,172.500000,172.500000,214496
2024-03-14 09:34:00-04:00,172.514999,173.009995,172.464996,172.839996,172.839996,382798
...,...,...,...,...,...,...
2024-03-22 15:56:00-04:00,172.375000,172.429993,172.330002,172.350006,172.350006,292582
2024-03-22 15:57:00-04:00,172.339996,172.419998,172.330002,172.404999,172.404999,347301
2024-03-22 15:58:00-04:00,172.409897,172.490005,172.389999,172.478897,172.478897,278936


In [33]:
data=yf.download(tickers='AAPL',start='2024-03-05',end='2024-03-12',interval='1m')

[*********************100%***********************]  1 of 1 completed


In [34]:
data


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-05 09:30:00-05:00,171.000000,171.000000,170.610001,170.919998,170.919998,5958861
2024-03-05 09:31:00-05:00,171.020004,171.074997,171.020004,171.059998,171.059998,926341
2024-03-05 09:32:00-05:00,171.335999,171.339996,171.270004,171.300003,171.300003,703908
2024-03-05 09:33:00-05:00,171.929993,171.970001,171.875000,171.895004,171.895004,994237
2024-03-05 09:34:00-05:00,171.625000,171.740005,171.580002,171.610001,171.610001,626087
...,...,...,...,...,...,...
2024-03-11 15:56:00-04:00,172.660004,172.779999,172.625000,172.770004,172.770004,317942
2024-03-11 15:57:00-04:00,172.770004,172.839996,172.720001,172.809998,172.809998,349009
2024-03-11 15:58:00-04:00,172.800003,172.807800,172.679993,172.729996,172.729996,408144
